# Train Model with XLA JIT Enabled

## Reference to All XLA Operations
https://www.tensorflow.org/performance/xla/operation_semantics

## Run the Next Cell to Show XLA JIT Training Code
Note the following device assignment:
```
with tf.device('device:XLA_GPU:0')
```

In [1]:
cat /root/src/main/python/xla/train_linear_xla.py

import tensorflow as tf
from tensorflow.python.client import timeline
import numpy as np

tf.reset_default_graph()

num_samples=100000

x_train = np.random.rand(num_samples).astype(np.float32)
print(x_train)

noise = np.random.normal(scale=0.01, size=len(x_train))

y_train = x_train * 0.1 + 0.3 + noise
print(y_train)

x_test = np.random.rand(len(x_train)).astype(np.float32)
print(x_test)

noise = np.random.normal(scale=0.01, size=len(x_train))

y_test = x_test * 0.1 + 0.3 + noise
print(y_test)

with tf.device("/cpu:0"):
  W = tf.get_variable(shape=[], name='weights')
  print(W)

  b = tf.get_variable(shape=[], name='bias')
  print(b)

  x_observed = tf.placeholder(shape=[None], dtype=tf.float32, name='x_observed')
  print(x_observed)

with tf.device("/job:localhost/replica:0/task:0/device:XLA_GPU:0"):
  y_pred = W * x_observed + b
  print(y_pred)

with tf.device("/job:localhost/replica:0/task:0/device:XLA_GPU:0"):
  y_observed = tf.placeholder(shape=[None], dtype=tf.float32, name='y_ob

## Run the Next Cell to Train with XLA Enabled
Ignore any errors you see.  This is OK.

In [4]:
%%bash 

cat /root/scripts/train_xla

#!/bin/bash

python /root/src/main/python/xla/train_linear_xla.py &> train_linear_xla.log

dot -T png /tmp/hlo_graph_1.*.dot -o /root/notebooks/hlo_graph_1.png &>/dev/null 

dot -T png /tmp/hlo_graph_10.*.dot -o /root/notebooks/hlo_graph_10.png &>/dev/null 

dot -T png /tmp/hlo_graph_25.*.dot -o /root/notebooks/hlo_graph_25.png &>/dev/null 

dot -T png /tmp/hlo_graph_50.*.dot -o /root/notebooks/hlo_graph_50.png &>/dev/null


In [5]:
%%bash

# IGNORE THE Aborted (cored dumped) Error!!
train_xla

/root/scripts/train_xla: line 3:  9436 Aborted                 (core dumped) python /root/src/main/python/xla/train_linear_xla.py &> train_linear_xla.log


## View Model Graph in Tensorboard

http://[ip-address]:6006

## View `hlo_graph_*` Files in Left Navigation
* `hlo_graph_*` **(JIT Visualizations)** 